# ISSAC 2023 Software Presentation of `dalgebra`

In this notebook we explain in more detail the software `dalgebra` as it was during its presentation in the International Symposium on Symbolic and Algebraic Computation (ISSAC) 2023 hold in Tromsø (Norway) between July 24th to 27th.

As a summary, `dalgebra` is a software for SageMath to represent and manipulate objects in the context of difference and differential algebra. In this context, we usually study rings $R$ where we have a set of associated operators $\Delta = \{\sigma_1,\ldots,\sigma_n\}$ that are additive endomorphisms, i.e., $\sigma_i : R \rightarrow R$ satisfies $\sigma_i(r+s) = \sigma_i(r)+\sigma_i(s)$. We call the pair $(R, \Delta)$ a **d-ring**.


## 1. Installation and documentation

The software `dalgebra` can be found in this repository. In particular, the version presented, the version 0.0.5, can be found [here](https://github.com/Antonio-JP/dalgebra/releases/tag/v0.0.5). In order to install it, simply run the following command line in a terminal where the command `sage` is available:

`sage -pip install [--user] git+https://github.com/Antonio-JP/dalgebra.git@v0.0.5`

where `[--user]` is an optional argument.

Once installed, we can load the software by importing `dalgebra`, and every functionality of the package will be available.

In [1]:
from dalgebra import *
%display latex

The documentation for the package at it latest version ca be found [here](https://antonio-jp.github.io/dalgebra/). 

## 2. The category of d-rings and creating from SageMath

The main feature (that will be used for future extension on the implementation) is the _Category_ (seen as in SageMath) of d-rings. This is implemented in the class `DRings`:

In [2]:
DRings()

Category of d rings

This category will provide any of its parent structures or element structures with enough methods to get the list of associated operators $\Delta$, apply any of these operators to its elements, etc. 

In general, we would like to take a ring structure from SageMath and attach to it some operators. This can be achieved using the factory `DRing`, which takes the following arguments:
* A SageMath structure for a ring $R$. It could also be a _d-ring_.
* A list of callables (or `Morphism`s) represting the set $\Delta$.
* (optionally) A list of types for the operators in $\Delta$. They define the callable as a `homomorphism`, a `derivation` or a `skew`-derivation. 

In order to provide clarity to the code, we provide shortcuts to the factory `DRing` with the methods `DifferentialRing` and `DifferenceRing` that call the factory with the corresponding type (`derivation` and `homomorphism`respectively).

As an example, we show now how to create the following rings:
1. $R = (\mathbb{Q}[x],\partial_x)$
2. $S = (\mathbb{Q}[c,x], (\partial_x, \sigma_x))$, where $\sigma_x(x) = x+1$.
3. $T = (\mathbb{Q}[x,y], (\partial_x, \partial_y, \sigma))$ where we define $\sigma(x) = x+1$ and $\sigma(y) = y-1$.

In [3]:
# Defining the first ring R
R = DifferentialRing(QQ[x], diff); R

Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]

In [4]:
# Defining the second ring S
B.<x,c> = QQ[]
S = DifferenceRing(DifferentialRing(B, lambda p : diff(p, x)), B.Hom(B)([x+1, c])); S

Ring [[Multivariate Polynomial Ring in x, c over Rational Field], (d/dx, Hom({x: x + 1, c: c}))]

In [5]:
# Defining the third ring T
B.<x,y> = QQ[]; dx,dy = B.derivation_module().gens()
T = DifferenceRing(DifferentialRing(B, dx, dy), B.Hom(B)([x+1, y-1])); T

Ring [[Multivariate Polynomial Ring in x, y over Rational Field], (d/dx, d/dy, Hom({x: x + 1, y: y - 1}))]

At this stage, we can create elements on each of these rings and see how the operation acts:

In [6]:
## Computing in the ring (QQ[x], dx)
x = R(x)
p = x^3 - 3*x + 2; show(p)
p.derivative()

x^3 - 3*x + 2

3*x^2 - 3

In [7]:
## Computing in the ring (QQ[x,c], (dx, sigma_x)
x = S(x); c = S(c)
p = c*x^2 - 3*x + c
show(r"p = " + latex(p))
show(r"\partial(p) = " + latex(p.derivative()))
show(r"\sigma(p) = " + latex(p.difference()))

p = x^{2} c - 3 x + c

\partial(p) = 2 x c - 3

\sigma(p) = x^{2} c + 2 x c - 3 x + 2 c - 3

In [8]:
## Computing in the ring (QQ[x,y], (dx, dy, sigma)
x = T(x); y = T(y)
p = x+y
show(r"p = " + latex(p))
show(r"\partial_x(p) = " + latex(p.derivative(0)))
show(r"\partial_y(p) = " + latex(p.derivative(1)))
show(r"\sigma(p) = " + latex(p.difference()))

p = x + y

\partial_x(p) = 1

\partial_y(p) = 1

\sigma(p) = x + y

We can check the types of the operators (and use then accordingly the method `derivative` and `difference`) by using the method `operator_types`:

In [9]:
R.operator_types()

('derivation',)

In [10]:
T.operator_types()

('derivation', 'derivation', 'homomorphism')

## 3. The ring of d-polynomials

Given a d-ring $(R, \Delta)$ we can always define what we call the ring of d-polynomials over $(R,\Delta)$ with a _d-variable_ $u$. This is a new d-ring with the same operations where we have taken as base ring the infinitely generated polynomial ring 
$$R[u_\alpha\ :\ \alpha \in \mathbb{N}^n]$$
where $|\Delta| = n$ and we extend the operations over $R$ by setting:
$$\sigma_i(u_\alpha) = u_{\alpha + e_i},$$
where $e_i$ is the $i$th canonical vector on $\mathbb{N}^n$. This can be then extended for several _d-variables_. We denote the new ring by $(R,\Delta)\{u\}$.

We can create this structure using `dalgebra` by the use of the _Factory_ `DPolynomialRing`. This factory only requires a valid d-ring and a list of names for the d-variables.

* The implementation is based on the class of infinite polynomials from SageMath. This means that the generators of the rings are not polynomials but the generators of variables.
* To ensure uniqueness in the structure, we sort the d-variables alphabetically. Hence $R\{u\}\{v\} \equiv R\{v\}\{u\} \equiv R\{u,v\}$.
* If another rings of d-polynomials is provided as argument, `DPolynomialRing` appends the new variables.

Following the examples above, we can construct easily the ring of d-polynomials over $S = (\mathbb{Q}[x,c], (\partial_x,\sigma_x))$ with d-variable $y$ with the following line:

In [11]:
DS.<y> = DPolynomialRing(S); DS

Ring of operator polynomials in (y) over Ring [[Multivariate Polynomial Ring in x, c over Rational Field], (d/dx, Hom({x: x + 1, c: c}))]

Now we can create any d-polynomial using the d-variable $y$. More precisely, when we want to create $\partial_x^a(\sigma_x^b(y)) \equiv y_{(a,b)}$, we only need to type $y[a,b]$:

In [12]:
y[0,0].derivative(), y[0,0].difference()

(y_1_0, y_0_1)

### Selected methods for d-polynomials and rings of d-polynomials

The d-polynomials have several incorporated functionalities beyond those already given as elements of a d-ring. Some selected are the following:

* `is_linear`: checks whether the d-polynomial is linear in any given set of d-variables. This means for any appearance of the given d-variables, they appear linearly. For example, the d-polynomials in $(\mathbb{Q},0)\{a,b\}$ are linear in $a$:
  $$a_0b_0^2 - a_1 + a_2b_2,\quad b_0b_1 - a_3b_2,$$
  but they are not linear in $b$ while the following polynomial is linear in both $a$ and $b$ but not in the set $(a,b)$:
  $$a_1 + b_1 -a_0b_0.$$
* `flatten`: allows to flatten a d-polynomial into a multivariable polynomial. This is helpful to take coefficients with respect to variables that are not d-variables.
* `eval`: given an element $t$ in any d-extension of $(R,\Delta)$, we can always evaluate a d-polynomial $(R,\Delta)\{y\}$ in the element $t$ where we substitute 
  $$y_{(\alpha_1,\ldots,\alpha_n)} \mapsto \left(\sigma_1^{\alpha_1}\circ\ldots\circ\sigma_n^{\alpha_n}\right)(t).$$
  This means, in particular, that we can evaluate a d-polynomial in another d-polynomial. This is specially helpful because it implements the composition of d-polynomials when seen as operators.
* `as_linear_operator`: when we have a d-polynomial in the d-variable $y$ and it is linear and homogenous, we can interpret the d-polynomials as a linear operator. Namely, the d-polynomial represents the action of the operator over an element. This methods transforms (if possible) a d-polynomial into an Ore polynomial that represents the same linear operator.
* `solve` (currently incomplete): given a d-variable appearing in the d-polynomial, this method tries to find an expression for said d-variable such that, when we call `eval` with that value, the d-polynomial evaluates to zero.

## 4. System of d-polynomials

In order to set up systems of d-polynomials, we provide a class with several methods that will allow to create, manipulate and compute with systems of d-polynomials. The class is called `DSystem`. This class receives:
* A list of d-polynomials that can be casted into a common ring of d-polynomials $(R, \Delta)\{y_1,\ldots,y_n\}$.
* A list of variables to be considered the unknowns of the system. They must be a subset of $\{y_1,\ldots, y_n\}$.

Following with the previously created ring $S = (\mathbb{Q}[x,c], (\partial_x,\sigma_x))$, we can define a system for the following two d-polynomials:

In [13]:
p1 = y[0,1] - y[0,0]
p2 = y[1,0] - c*y[0,0]
S = DSystem([p1,p2]); S

System over [Ring of operator polynomials in (y) over Ring [[Multivariate Polynomial Ring in x, c over Rational Field], (d/dx, Hom({x: x + 1, c: c}))]] with variables [(y_*,)]:
{
	y_0_1 - y_0_0 == 0
	y_1_0 + (-c)*y_0_0 == 0
}

### Selected methods for system of d-polynomials

The clas `DSystem` includes several methods to manipulate systems of d-polynomials. Some of these methods are:
* `extend_by_operation`: if $p(y) = 0$, then $\sigma(p(y)) = 0$ no matter which $\sigma$ we have defined. Hence, given a system of d-polynomials, we can always compute more equations that are equal to zero by applying one operation to each polynomial. This method automatizes this procedure, creating a system with more equations. This is crucial to many elimination algorithms.
* There are several methods to obtain subsystems of equations. In particular, the slice notation in Python allows to get a subsystem for a given set of indices.
* `is_homogeneous`: checks whether all equations in the system are homogeneous (when looked as algebraic polynomials).
* `is_linear`: check whether the system is linear in a given set of variables.
* `solve_linear`: if the system is linear in its variables, this method tries to solve the system obtaining a dictionary of solutions such that, if we evaluate all the equations using this dictionary (see method `eval`) we obtain always zero.
* `diff_resultant`: method to compute the differential/difference resultant of a system. This will allow to eliminate $n$ d-variables from a set of $n-1$ d-polynomial equations. Currently this implementation is incomplete.

## 5. Examples of use (section 3 of software presentation)

### 5.1 Computing a commuting operator

Assume that we are considering the linear differential operator $L_2 = \partial^2 - u$ where $u$ is a d-variable. We want to compute an operator of order 3 of the form $P_3 = \partial^3 + a\partial + b$ (for some $a$ and $b$) such that $L_2$ and $P_3$ commute. This means that $[L,P] = LP - PL = 0$.

We can get the system for $a$ and $b$ using `dalgebra`. For doing so, we first create a ring of d-polynomials with variables $u$, $a$ and $b$. We also add the d-variable $y$ to include the operator $\partial$ as a d-polynomial, since:
$$L_2\cdot y = y'' - uy,\qquad P_3\cdot y = y''' + ay' + by.$$

In [14]:
R.<a,b,u,y> = DPolynomialRing(DifferentialRing(QQ, lambda p:0))
L = y[2] - u[0]*y[0]; P = y[3] + a[0]*y[1] + b[0]*y[0]

We can the proceed to compute the commutator of these two operators. This can be done by evaluating the d-variable $y$ in the other operator:

In [15]:
C = L(y=P) - P(y=L); C

a_0*u_1*y_0 + a_2*y_1 + 2*a_1*y_2 + b_2*y_0 + 2*b_1*y_1 + u_3*y_0 + 3*u_2*y_1 + 3*u_1*y_2

We can observe that, in general, the operators $L_2$ and $P_3$ do not commute since $C = [L,P] \neq 0$. If we want to compute a commutator, we need that all the coefficients w.r.t. $y$ must vanish. We can get this system by extracting coefficients from $C$:

In [16]:
S = DSystem([C.coefficient(y[i]) for i in range(C.order(y)+1)], variables=[a,b]); S

System over [Ring of operator polynomials in (a, b, u, y) over Differential Ring [[Rational Field], (0,)]] with variables [(a_*, b_*)]:
{
	a_0*u_1 + b_2 + u_3 == 0
	a_2 + 2*b_1 + 3*u_2 == 0
	2*a_1 + 3*u_1 == 0
}

In this particular case, we end up with 3 equations and two variables. This usually has no solution and depends on the preoperties of the function $u$. We can get this condition on $u$ by solving the subsystem on $a$ and $b$ in the last two equations and pluging their value into the first equation:

In [17]:
sol_ab = S[1:].solve_linear(); sol_ab

{a_*: (-3/2)*u_0, b_*: (-3/4)*u_1}

In [18]:
S.equation(0)(dic=sol_ab)

(-3/2)*u_1*u_0 + 1/4*u_3

### 5.2 Sylvester resultant and subresultants

Another computation that is available is the computation of the Sylvester resultant of two linear d-polynomials and the subresultant sequence. Assume we have the following operators:
$$L_1 = \partial^2 - (\cosh(x) + e^x) \partial + e^x(\cosh(x) -1),\qquad L_2 = \partial^2 - (\sinh(x) + e^x) \partial + e^x(\sinh(x) -1).$$
If we would like to know if they share a common solution, we would need to compute the resultant of the two operators.

This can be done in `dalgebra` using the method `sylvester_resultant`. First, we need to define the ring of d-polynomials to represent $L_1$ and $L_2$. In order to do so, we need to include in the system the $\cosh(x)$ and $\sinh(x)$. This requires the exponential function $e^x$ to be added in the system:

In [19]:
B.<ex> = QQ[] # ex is e^x
F = DifferentialRing(B.fraction_field(), lambda ex: ex); show(F)
F(ex).operation()

Differential Ring [[Fraction Field of Univariate Polynomial Ring in ex over Rational Field], (ex*d/dex,)]

ex

Then we proceed to define the $\cosh(x)$ and $\sinh(x)$:

In [20]:
cosh = F((ex + (1/ex))/2)
sinh = F((ex - (1/ex))/2)
ex = F(ex)
cosh.operation() == sinh and sinh.operation() == cosh

True

We then proceed to create the ring of d-polynomials necessary to represent the two linear operators $L_1$ and $L_2$. Then we can simply compute the Sylvester resultant:

In [21]:
R.<y> = DPolynomialRing(F)
L1 = y[2] - (cosh + ex)*y[1] + ex*(cosh-F(1))*y[0]
L2 = y[2] - (sinh + ex)*y[1] + ex*(sinh-F(1))*y[0]
L1.sylvester_resultant(L2)

0

Hence they share a common solution. In order to see which one, we could compute the right greatest common divisor of $L_1$ and $L_2$. We can do it by checking the first non-zero subresultant operator. This can be done by the method `sylvester_subresultant_sequence` in the ring of d-polynomials:

In [22]:
R.sylvester_subresultant_sequence(L1, L2)

(0, (-1/ex)*y_1 + y_0)

This means that the solution to the differential operator $\partial - e^x$ (i.e., $ce^{e^x}$) is a common solution for both $L_1$ and $L_2$.

### 5.3 Elimination of difference variables

In this last example we are going to show how, from a non-linear difference system, we can eliminate one of the variables using \packname. Consider the following difference system:
$$\label{equ:difference_system}\left\{\begin{array}{ll}
    u(n+1) - 2^n u(n)^2 + u(n)v(n) & = 0,\\
    v(n+1) - 3^n v(n)^2 + u(n)v(n) & = 0,
\end{array}\right.$$
and assume we want to eliminate the variable $u(n)$ to obtain a relation only involving $v(n)$ that all solutions to the system satisfy.

We can do this in \packname by computing the difference resultant of the system. We start by creating the elements representing the two exponential sequences $2^n$ and $3^n$:

In [23]:
B.<e2,e3> = QQ[]
DB = DifferenceRing(B, B.Hom(B)([2*e2, 3*e3])) # e2 represents 2^n and e3 represents 3^n
e2, e3 = DB(e2), DB(e3)

As an example, we have that $\sigma(2^n3^n) = \sigma(6^n) = 6\cdot 6^n$:

In [24]:
(e2*e3).shift()//(e2*e3)

6

Now we proceed to create two difference variables $u(n),v(n)$ and the two equations of the system above:

In [25]:
R.<u,v> = DPolynomialRing(DB)
f1 = u[1] - e2*u[0]^2 + u[0]*v[0]
f2 = v[1] - e3*v[0]^2 + u[0]*v[0]

The system of d-equations can now be created. Since we want to eliminate the $u$, we set this as the variable for the system:

In [26]:
S = DSystem([f1,f2], variables=[u]); S

System over [Ring of operator polynomials in (u, v) over Difference Ring [[Multivariate Polynomial Ring in e2, e3 over Rational Field], (Hom({e2: 2*e2, e3: 3*e3}),)]] with variables [(u_*,)]:
{
	(-e2)*u_0^2 + u_0*v_0 + u_1 == 0
	u_0*v_0 + (-e3)*v_0^2 + v_1 == 0
}

Finally, we can call the method `diff_resultant` to eliminate the $u$. The obtained result is a relation for the $v$:

In [27]:
S.diff_resultant()

(e2*e3^2 - e3)*v_1*v_0^4 + (-2*e2*e3 - 3*e3 + 1)*v_1^2*v_0^2 + v_2*v_0^2 + e2*v_1^3

Rewriting this in their meaning as sequences, we have obtained that $v(n)$ satisfies:
$$3^n(6^n - 1) v(n+1)v(n)^4 - (2\cdot 6^n + 3^{n+1} - 1)v(n+1)^2v(n)^2 + v(n+2)v(n)^2 + 2^nv(n+1)^3 = 0$$

From this equation we can get an expression for $v(n+2)$ in terms of the previous values:
$$v(n+2) = \frac{(2\cdot 6^n + 3^{n+1} - 1)v(n+1)^2v(n)^2 - 3^n(6^n - 1) v(n+1)v(n)^4 - 2^nv(n+1)^3}{v(n)^2},$$
and if we want $u(0) = 2$ and  $v(0) = 1$, we then have $v(1)=1$. Hence we can unroll the sequence $v(n)$:

In [28]:
from functools import lru_cache
@lru_cache(maxsize=None)
def v(n):
    if n == 0: return 1
    elif n == 1: return 1
    else:
        numerator = (2*6**(n-2) + 3**(n-1) - 1)*v(n-1)**2*v(n-2)**2 - 3**(n-2)*(6**(n-2)-1)*v(n-1)*v(n-2)**4 - 2**(n-2)*v(n-1)**3
        denominator = v(n-2)**2
        return numerator/denominator

In [29]:
[v(i) for i in range(6)]

[1, 1, 3, 81, 177147, 2541865828329]

Similarly, we can then obtain $u(n)$ from the first equation:

In [30]:
@lru_cache(maxsize=None)
def u(n):
    if n == 0: return 2
    else:
        return 2**(n-1)*u(n-1)**2 - u(n-1)*v(n-1)

In [31]:
[u(i) for i in range(6)]

[2, 2, 6, 126, 116802, 197592191370]